## Install Packages

In [1]:
!pip show s3fs psycopg2 rpy2 hdfs

Name: s3fs
Version: 0.4.2
Summary: Convenient Filesystem interface over S3
Home-page: http://github.com/dask/s3fs/
Author: None
Author-email: None
License: BSD
Location: /opt/conda/lib/python3.7/site-packages
Requires: fsspec, botocore
Required-by: 
---
Name: psycopg2
Version: 2.8.5
Summary: psycopg2 - Python-PostgreSQL Database Adapter
Home-page: https://psycopg.org/
Author: Federico Di Gregorio
Author-email: fog@initd.org
License: LGPL with exceptions
Location: /opt/conda/lib/python3.7/site-packages
Requires: 
Required-by: 
---
Name: rpy2
Version: 3.3.2
Summary: Python interface to the R language (embedded R)
Home-page: https://rpy2.github.io
Author: Laurent Gautier
Author-email: lgautier@gmail.com
License: GPLv2+
Location: /opt/conda/lib/python3.7/site-packages
Requires: pytest, pytz, jinja2, cffi, tzlocal
Required-by: 
---
Name: hdfs
Version: 2.5.8
Summary: HdfsCLI: API and command line interface for HDFS.
Home-page: https://hdfscli.readthedocs.io
Author: Matthieu Monsch
Author-ema

In [2]:
#general
import pandas as pd
import os

#s3 python utility
import s3fs

#postgres python utility
import psycopg2 as pg
import pandas.io.sql as psql

#hdfs client python utility
from hdfs import InsecureClient

## Cloud data access: Pull sample of 2016 data from s3

Note: s3fs automatically checks aws_access_key_id, aws_secret_access_key, and aws_session_token environment variables. 

In Domino, we can securely store these env variables at the user or project level, e.g., 
* os.environ["AWS_ACCESS_KEY_ID"]
* os.environ["AWS_SECRET_ACCESS_KEY"]

In [8]:
dfs3 = pd.read_csv('s3://seteam-files/mq/2016.csv', nrows=100000)
dfs3.describe()
#dfs3.head(10)

,OP_CARRIER_FL_NUM,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
count,100000.00000,100000.000000,99165.000000,99165.000000,99142.000000,99142.000000,99055.000000,99055.000000,100000.000000,99055.000000,...,100000.000000,98944.000000,98944.000000,100000.000000,18045.000000,18045.000000,18045.000000,18045.000000,18045.000000,0.0
mean,2281.12279,1341.278570,1345.923451,10.002783,16.177987,1365.601622,1471.261915,7.534854,1493.719450,1474.247378,...,147.803680,140.960836,117.250889,857.919580,20.553228,1.596453,12.461457,0.117761,22.070490,NaN
std,1885.95176,495.384206,507.392091,37.832780,8.403221,509.321488,536.905038,5.999436,525.445745,542.205617,...,77.347018,75.041098,72.982850,616.921346,54.884938,15.896559,27.275072,2.043133,42.031081,NaN
min,1.00000,1.000000,1.000000,-46.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,21.000000,15.000000,8.000000,31.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
25%,738.00000,923.000000,925.000000,-4.000000,11.000000,939.000000,1052.000000,4.000000,1110.000000,1056.000000,...,90.000000,85.000000,63.000000,399.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
50%,1728.00000,1335.000000,1342.000000,-1.000000,14.000000,1354.000000,1516.000000,6.000000,1525.000000,1519.000000,...,130.000000,123.000000,99.000000,692.000000,4.000000,0.000000,1.000000,0.000000,3.000000,NaN
75%,3483.00000,1740.000000,1750.000000,9.000000,19.000000,1803.000000,1918.000000,8.000000,1925.000000,1922.000000,...,181.000000,174.000000,148.000000,1105.000000,20.000000,0.000000,16.000000,0.000000,28.000000,NaN
max,7438.00000,2359.000000,2400.000000,1333.000000,147.000000,2400.000000,2400.000000,133.000000,2359.000000,2400.000000,...,705.000000,701.000000,683.000000,4983.000000,1313.000000,898.000000,755.000000,123.000000,1109.000000,NaN


## Legacy DB access: Pull sample of 2017 data from Postgres

* os.environ['user'] and os.environ['db_password'] stored in project env vars

In [9]:
dbname = 'postgres'

connection = pg.connect("host='10.10.0.5' dbname="+dbname+" user="+os.environ['user'])

dfpg = pd.read_sql_query('select * from import.airline_2017 limit 100000',con=connection)
dfpg.columns = dfpg.columns.str.upper()
dfpg.describe()
#dfpg.head(10)

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,UNNAMED__27
count,100000,100000,100000,100000,100000,100000,100000,100000,100000,100000,...,100000,100000,100000,100000,100000,100000,100000,100000,100000,100000
unique,8,12,5774,292,292,1115,1360,518,126,1362,...,426,520,508,1223,406,150,224,31,312,1
top,2017-01-27,WN,311,ATL,ATL,600,,-5.0,12.0,,...,80.0,,,337.0,,,,,,\r
freq,15436,24043,85,6701,6793,1826,870,8764,7883,906,...,2278,1098,1098,937,84063,84063,84063,84063,84063,100000


## Pull all 2018 from HDFS
* Connection string managed from within Domino Environment Variable os.environ['hdfs_conn']

In [10]:
client_hdfs = InsecureClient(os.environ['hdfs_conn'])

In [12]:
# ====== Reading files ======
with client_hdfs.read('/data/2018.csv', encoding = 'utf-8') as reader:
    dfhdfs = pd.read_csv(reader,nrows=100000)
dfhdfs.describe()
#dfhdfs.head(10)

,OP_CARRIER_FL_NUM,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
count,100000.000000,100000.000000,93437.000000,93322.000000,93376.000000,93376.000000,93282.000000,93282.000000,100000.000000,93282.000000,...,100000.000000,93150.000000,93150.000000,100000.000000,22437.000000,22437.000000,22437.000000,22437.000000,22437.000000,0.0
mean,2589.082050,1335.709640,1349.856930,16.478505,17.394491,1372.316056,1466.432902,7.875678,1485.803180,1469.092140,...,144.739340,137.700676,112.439452,811.589560,25.435798,4.364175,10.201854,0.165396,28.391095,NaN
std,1960.645215,497.746561,511.545158,55.314596,10.020054,512.503337,543.778205,6.897298,528.949398,549.407003,...,75.335381,73.979704,71.890603,603.696509,68.537332,34.407350,27.541732,3.523026,55.168089,NaN
min,1.000000,1.000000,1.000000,-49.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,23.000000,20.000000,9.000000,31.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
25%,858.000000,915.000000,927.000000,-4.000000,11.000000,941.000000,1049.000000,4.000000,1100.000000,1053.000000,...,90.000000,84.000000,60.000000,366.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
50%,1954.000000,1327.000000,1341.000000,0.000000,15.000000,1354.000000,1511.000000,6.000000,1517.000000,1514.000000,...,126.000000,119.000000,93.000000,646.000000,6.000000,0.000000,0.000000,0.000000,6.000000,NaN
75%,4221.000000,1741.000000,1755.000000,15.000000,20.000000,1809.000000,1916.000000,9.000000,1921.000000,1921.000000,...,176.000000,169.000000,143.000000,1050.000000,24.000000,0.000000,14.000000,0.000000,34.000000,NaN
max,7439.000000,2359.000000,2400.000000,1752.000000,164.000000,2400.000000,2400.000000,258.000000,2359.000000,2400.000000,...,704.000000,728.000000,637.000000,4983.000000,1752.000000,1682.000000,1317.000000,264.000000,1648.000000,NaN


## Merge/join s3 & Postgres & HDFS

In [13]:
#dfpg_2 = dfpg_2.to_numeric('float64',errors='ignore').dtypes
#frames = [dfs3_2, dfpg_2]
merged = pd.concat([dfs3,dfhdfs],axis=0, sort=False, ignore_index=True)
merged = pd.concat([merged,dfpg],axis=0, sort=False, ignore_index=True)
#merged = merged.apply(pd.to_numeric(errors='ignore'))
#merged = dfs3_2.append(dfpg_2)
#pd.to_numeric(merged, errors='ignore')
merged = merged.apply( pd.to_numeric, errors='ignore' )
merged.describe()
#merged.head(10)

,OP_CARRIER_FL_NUM,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
count,300000.000000,300000.000000,291732.000000,291617.000000,291612.000000,291612.000000,291388.000000,291388.000000,300000.000000,291388.000000,...,299999.000000,290996.000000,290996.000000,300000.000000,56419.000000,56419.000000,56419.000000,56419.000000,56419.000000,0.0
mean,2292.175853,1333.852827,1340.292354,11.276990,16.777639,1362.224391,1473.126409,7.616261,1496.001190,1477.102025,...,147.008010,140.446697,116.058035,841.753387,23.853684,2.728301,12.284638,0.118861,24.678264,NaN
std,1861.714363,490.807012,503.292617,45.354085,9.227348,504.421270,530.653448,6.489899,517.678229,535.557294,...,76.645138,74.959923,72.969095,613.169222,64.115322,25.330310,27.845366,2.601599,49.071282,NaN
min,1.000000,1.000000,1.000000,-56.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,21.000000,15.000000,7.000000,31.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
25%,741.000000,920.000000,923.000000,-5.000000,11.000000,938.000000,1055.000000,4.000000,1110.000000,1059.000000,...,90.000000,85.000000,62.000000,387.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
50%,1734.000000,1329.000000,1336.000000,-1.000000,14.000000,1348.000000,1514.000000,6.000000,1525.000000,1517.000000,...,129.000000,122.000000,97.000000,678.000000,4.000000,0.000000,1.000000,0.000000,2.000000,NaN
75%,3680.000000,1735.000000,1745.000000,9.000000,20.000000,1759.000000,1915.000000,9.000000,1923.000000,1920.000000,...,180.000000,173.000000,147.000000,1080.000000,21.500000,0.000000,16.000000,0.000000,30.000000,NaN
max,7439.000000,2359.000000,2400.000000,1752.000000,168.000000,2400.000000,2400.000000,258.000000,2359.000000,2400.000000,...,712.000000,728.000000,683.000000,4983.000000,1752.000000,1682.000000,1317.000000,264.000000,1648.000000,NaN


# Export to a Domino Dataset
* provides shared, versioned, reproducible access to data

In [15]:
merged.to_csv(r'/domino/datasets/goyetc/data-access/scratch/airlines_2016_2017_2018.csv', index = None, header=True) 